In [2]:
import os
cwd = os.getcwd()
cwd
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
import pandas as pd 
from glob import glob
import PIL.Image as pil 
import PIL.ImageOps
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet

from imutils import paths

import argparse
import random
import pickle
import cv2

 

C:\Users\mahmo\Anaconda3\lib\site-packages\matplotlib\__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# Convert image folders into RGB format , resize it to 256 and save it into another foldr

In [3]:
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def crop_and_resize_images(path, new_path, img_size):
    create_directory(new_path)
    dirs = [l for l in os.listdir(path) if l != '.DS_Store']
    

    for item in dirs:
        # Read in all images  
        img = cv2.imread(path + item)
        img = cv2.resize(img, (img_size, img_size))
        cv2.imwrite(str(new_path + item), img)

# Call above function to create RGB , Resize and Save it 

In [4]:
#crop_and_resize_images(path='D:/Udacity/Sub3_dog_breed/Xrayimages_004/images/', new_path='D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/', img_size=256)   

In [5]:
CSV = pd.read_csv("D:/Udacity/Sub3_dog_breed/Xrayimages_004/Data_Entry_2017/Data_Entry_2017.csv")
sample=CSV[['Image Index','Finding Labels']]



# Read File Names

In [6]:
filespathTrain="D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages"


In [7]:
def get_lst_image_names(file_path):
    """
    Reads in all files from file path into a list.
    INPUT
        file_path: specified file path containing the images.
    OUTPUT
        List of image strings
    """
    list_img_names=[i for i in os.listdir(file_path) if i != '.DS_Store']
    return pd.DataFrame({'Image Index' :list_img_names})




In [8]:
list_img_names_train=get_lst_image_names(filespathTrain)

#len(list_img_names)
#Image_index_Df = pd.DataFrame({'Image Index' :list_img_names})

list_img_names_train.head()


,Image Index
0,00006585_007.png
1,00006585_008.png
2,00006585_009.png
3,00006585_010.png
4,00006585_011.png


# MargeCSV with File Names

In [9]:
def mergfun(Image_index_Df):
    Image_labelsDf = pd.merge(Image_index_Df, sample, how='left', on='Image Index')
    return Image_labelsDf



In [10]:
Image_labelsTrainDf=mergfun(list_img_names_train)
Image_labelsTrainDf.head()


,Image Index,Finding Labels
0,00006585_007.png,Atelectasis
1,00006585_008.png,No Finding
2,00006585_009.png,Infiltration
3,00006585_010.png,Infiltration|Mass
4,00006585_011.png,Infiltration|Pneumothorax


In [11]:
Image_labelsTrainDf.count()

Image Index       10000
Finding Labels    10000
dtype: int64

# Preprocess CSV file

In [12]:
Image_labelsTrainDf.count()

Image Index       10000
Finding Labels    10000
dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
Image_labelsTrainDf.count()

Image Index       10000
Finding Labels    10000
dtype: int64

In [15]:
image_train,image_valid,label_train, label_valid=train_test_split(Image_labelsTrainDf['Image Index'],Image_labelsTrainDf['Finding Labels'],test_size=0.2)

In [16]:
print (image_train.shape)
print(label_train.shape)
print( image_valid.shape)
print(label_valid.shape)

(8000,)
(8000,)
(2000,)
(2000,)


In [18]:
label_train.head()


8612           Pneumothorax
5877             No Finding
2958    Infiltration|Nodule
121              No Finding
1123           Infiltration
Name: Finding Labels, dtype: object

In [19]:
image_train.shape

(8000,)

In [20]:
label_train.shape

(8000,)

In [21]:
image_valid.shape

(2000,)

In [22]:
label_valid.shape

(2000,)

# Print the list of multiple Lables in all datasets

In [24]:
def printLables(label):
    for (i, label) in enumerate(mlb.classes_):
         print("{}. {}".format(i + 1, label))
        

# To find one hot vector for multiple Lables for each class

In [25]:
#Image_labelsTrainDf
mlb = MultiLabelBinarizer() 
def MultiLabelTarget(DF):
    labels = []
    labels=DF.apply(lambda x: x.split('|'))
    labels = mlb.fit_transform(labels)
    printLables(labels)
    return labels



In [66]:
tranTarget1=MultiLabelTarget(label_train)

1. Atelectasis
2. Cardiomegaly
3. Consolidation
4. Edema
5. Effusion
6. Emphysema
7. Fibrosis
8. Hernia
9. Infiltration
10. Mass
11. No Finding
12. Nodule
13. Pleural_Thickening
14. Pneumonia
15. Pneumothorax


from itertools import count

class Obj(object):
  _ids = count(0)

  def __init__(self):
    self.id = next(self._ids)

NameError: name 'Atelectasis' is not defined

In [30]:


ValidTarget1=MultiLabelTarget(label_valid)

1. Atelectasis
2. Cardiomegaly
3. Consolidation
4. Edema
5. Effusion
6. Emphysema
7. Fibrosis
8. Hernia
9. Infiltration
10. Mass
11. No Finding
12. Nodule
13. Pleural_Thickening
14. Pneumonia
15. Pneumothorax


In [68]:
tranTarget1[0:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [31]:
Atelectasis.id

NameError: name 'Atelectasis' is not defined

In [69]:
ValidTarget1[0:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [46]:
len(ValidTarget1)

2000

In [47]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


In [48]:
def prepareListOfImageName(listOfFlile, pathRGB): 
    listOfFlile=listOfFlile
    #listOfFlile[0:10]
    l=[]
    for filename in listOfFlile:
        l.append(pathRGB+'/'+filename) 
        #l.append('D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/'+filename)
    return np.array(l)    
  

In [49]:
trainImgeNameArray1=prepareListOfImageName(image_train, filespathTrain)

In [50]:
trainImgeNameArray1[0:15]

array(['D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00006625_000.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00007514_002.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00007935_002.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008295_042.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00006948_014.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008515_001.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008701_011.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008830_001.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00007241_001.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008250_000.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008875_020.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00007093_017.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008

In [51]:
ValidImgeNameArray1=prepareListOfImageName(image_valid,filespathTrain)

In [52]:
ValidImgeNameArray1[0:5]

array(['D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008172_000.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008278_000.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008456_002.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00008289_000.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00006978_004.png'], 
      dtype='<U67')

In [53]:
train_tensors1 = paths_to_tensor(trainImgeNameArray1).astype('float32')/255;

100%|█████████████████████████████████████████████████████████████████████████████| 8000/8000 [00:19<00:00, 411.97it/s]


In [54]:
valid_tensors1 = paths_to_tensor(ValidImgeNameArray1).astype('float32')/255;

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:39<00:00, 50.95it/s]


In [55]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
print(len(train_tensors1))
print(train_tensors1.shape)
print(len(tranTarget1))
print(len(valid_tensors1))
print(len(ValidTarget1))
#import random
#random.seed(8675309)

model = Sequential()

### Define architecture.

model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu',input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))

model.add(Dense(15, activation='sigmoid'))
model.summary()

8000
(8000, 224, 224, 3)
8000
2000
2000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 

In [56]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint  

###  specify the number of epochs that you would like to use to train the model.

epochs = 10

###   the code below set the chechpoints and save the model

#checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, tranTarget, 
          validation_data=(valid_tensors, validTarget),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

In [57]:
epochs = 5




model.fit(train_tensors1,tranTarget1, 
          validation_data=(valid_tensors1 , ValidTarget1),
          epochs=epochs, batch_size=20,  verbose=1)

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - ETA: 435s - loss: 3.8501 - acc: 0.0500 - ETA: 319s - loss: 3.5364 - acc: 0.1500 - ETA: 288s - loss: 3.4196 - acc: 0.2333 - ETA: 274s - loss: 3.3779 - acc: 0.3500 - ETA: 270s - loss: 3.3725 - acc: 0.3900 - ETA: 268s - loss: 3.2880 - acc: 0.4250 - ETA: 266s - loss: 3.1815 - acc: 0.4714 - ETA: 263s - loss: 3.1739 - ac